In [1]:
import torch
import numpy as np
import pandas as pd


# read whole corpus
processed_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/processed_df.csv')

# work on subset of speakers
temp_bioguides = ['H001055','C001074']
temp_df = processed_df[processed_df.bioguide_id.isin(temp_bioguides)]


/cluster/scratch/goezsoy/miniconda3/envs/cond_text_gen_project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
temp_df.loc[87084].speech

". Mr. Speaker, I rise today to recognize the outstanding career of Dr. John W. Becher, or ``Chief'' as he was called by scores of medical residents, an osteopathic physician who has dedicated his life to his patients, his students, and to the improvement of the medical profession.  Dr. Becher's commitment to the field of emergency medicine spans more than 30 years. As professor and chairman of the Department of Emergency Medicine at the Philadelphia College of Osteopathic Medicine, he has helped countless students and residents, myself included, develop their skills and become an essential part of our health care workforce.  As a young resident at Albert Einstein Medical Center, I was fortunate to have Dr. Becher's insight and guidance as my residency director. His dedication to emergency medicine was evident then, and his understanding of the osteopathic profession was invaluable to my training and to my career.  His involvement in the field of osteopathic medicine is unparalleled. I

In [5]:
# loading model for generation
from model import Net

# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

model = Net().to(device)

checkpoint = torch.load('../checkpoints/speaker_name_as_prompt_deneme_best.pt',map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

cpu


Net(
  (gpt_neo): GPTNeoForCausalLM(
    (transformer): GPTNeoModel(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(2048, 768)
      (drop): Dropout(p=0, inplace=False)
      (h): ModuleList(
        (0): GPTNeoBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPTNeoAttention(
            (attention): GPTNeoSelfAttention(
              (attn_dropout): Dropout(p=0, inplace=False)
              (resid_dropout): Dropout(p=0, inplace=False)
              (k_proj): Linear(in_features=768, out_features=768, bias=False)
              (v_proj): Linear(in_features=768, out_features=768, bias=False)
              (q_proj): Linear(in_features=768, out_features=768, bias=False)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPTNeoMLP(
            (c_fc): Linear(in_features=768, out_features=3072,

In [22]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token

inputs = tokenizer('Michael Jordan says', return_tensors="pt")

gen_tokens = model.gpt_neo.generate(
    inputs.input_ids,
    do_sample=True,
    temperature=0.4,
    max_length=100,
)
tokenizer.batch_decode(gen_tokens)[0]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"Michael Jordan says: “I want to thank the gentleman from across the aisle for his kind remarks.  Mr. Speaker, I rise today to recognize the outstanding career of Dr. John W. Becher, or ``Chief'' as he was called by scores of medical residents, an osteopathic physician who has dedicated his life to his patients, his students, and to the improvement of the medical profession.  Dr. Becher's commitment to the field of emergency medicine spans more than 30 years"